# Combinations vectorized

#### Uses the following information:
- parents of pairs
- pairs indices
- Usual starts,stops,counts etc.

#### Basic idea:

We can note that `left` is the quotient of division between `pairs_index-pairs_start[event]` and `counts[event]`. `right` on the other hand, is the result of modulus of the abovementioned factors. We can use it to form the result.


In [30]:
import numpy as np
import sys
import numba
from awkward import *

In [31]:
NUMEVENTS = 50
AVENUMJETS = 2

numjets1 = np.random.poisson(AVENUMJETS, NUMEVENTS).astype(np.int)
stops1 = np.cumsum(numjets1).astype(np.int)
starts1 = np.zeros_like(stops1)
starts1[1:] = stops1[:-1]

counts1 = stops1-starts1
offsets1 = np.zeros(len(numjets1)+1)
offsets1[1:] = stops1[:]

numjets2 = np.random.poisson(AVENUMJETS, NUMEVENTS).astype(np.int)
stops2 = np.cumsum(numjets2).astype(np.int)
starts2 = np.zeros_like(stops2)
starts2[1:] = stops2[:-1]


counts2 = stops2-starts2
offsets2 = np.zeros(len(numjets2)+1)
offsets2[1:] = stops2[:]

In [32]:
parents = np.empty(stops[-1], dtype=np.int)

@numba.jit()
def vectorized_search(offsets, content):
    index = np.arange(len(content), dtype=int)                     # threadIdx.x on CUDA
    below = np.zeros(len(content), dtype=int)                      # just below = 0 on CUDA
    above = np.ones(len(content), dtype=int) * (len(offsets) - 1)  # same for above
    while True:
        middle = (below + above) // 2

        change_below = offsets[middle + 1] <= index                   # which "belows" must we change?
        change_above = offsets[middle] > index                        # which "aboves"?

        if not np.bitwise_or(change_below, change_above).any():    # neither? great! we're done!
            break
        else:
            below = np.where(change_below, middle + 1, below)      # vectorized "if" statement
            above = np.where(change_above, middle - 1, above)      # this is the only branch

    return middle

In [33]:
# pairs_indices should properly be called pairs_counts
pairs_indices = np.zeros(NUMEVENTS+1)
pairs_indices[1:] = np.cumsum(counts1*counts2)
pairs_indices = pairs_indices.astype(np.int)

In [34]:
# pairs_contents should be called pairs_indices
pairs_contents = np.arange(pairs_indices[-1]).astype(np.int)
pairs_parents = vectorized_search(pairs_indices, pairs_contents)
pairs_parents = pairs_parents.astype(np.int)

In [35]:
left = np.empty_like(pairs_contents)
right = np.empty_like(pairs_contents)

In [36]:
left[pairs_contents] = starts1[pairs_parents[pairs_contents]] + np.floor((pairs_contents-pairs_indices[pairs_parents[pairs_contents]])/counts2[pairs_parents[pairs_contents]]).astype(np.int)
right[pairs_contents] = starts2[pairs_parents[pairs_contents]]+(pairs_contents-pairs_indices[pairs_parents[pairs_contents]])-counts2[pairs_parents[pairs_contents]]*np.floor((pairs_contents-pairs_indices[pairs_parents[pairs_contents]])/counts2[pairs_parents[pairs_contents]])

In [41]:
# Randdom Content
for i in range(6):
    print("Event {}\n Left {}\nRight {}\n\n".format(i, left[pairs_indices[i]:pairs_indices[i+1]], right[pairs_indices[i]:pairs_indices[i+1]]))

Event 0
 Left [0 0 0 0 1 1 1 1]
Right [0 1 2 3 0 1 2 3]


Event 1
 Left [2 2 3 3]
Right [4 5 4 5]


Event 2
 Left [4 4 4 5 5 5]
Right [6 7 8 6 7 8]


Event 3
 Left [6]
Right [9]


Event 4
 Left [7 7 7 7]
Right [10 11 12 13]


Event 5
 Left []
Right []




In [42]:
# Contents
content1 = np.random.randn(stops[-1])
content2 = np.random.randn(stops[-1])

In [43]:
# JaggedArrays
arr1 = JaggedArray.fromoffsets(pairs_indices.astype(np.int), content1[left])
arr2 = JaggedArray.fromoffsets(pairs_indices.astype(np.int), content2[right])

In [45]:
arr1

<JaggedArray [[ 0.09501511  0.09501511  0.09501511  0.09501511 -0.66320606 -0.66320606
 -0.66320606 -0.66320606] [ 1.56090746  1.56090746 -1.05798713 -1.05798713] [ 0.78628129  0.78628129  0.78628129 -0.03873091 -0.03873091 -0.03873091] ... [-0.47996288 -0.47996288 -0.47996288 -0.47996288], [-0.69266046 -0.69266046 -1.4222636  -1.4222636 ], [ 1.35448842  1.35448842  1.35448842  1.35448842 -1.174081   -1.174081
 -1.174081   -1.174081  ]] at 0201adce7208>

#### How to combine this JaggedArrays()?